<a href="https://colab.research.google.com/github/uakarsh/TiLT-Implementation/blob/main/how_did_i_prepare_the_stuffs/tilt_part_2_1_preparing_the_visual_model_funsd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/uakarsh/TiLT-Implementation.git

Cloning into 'TiLT-Implementation'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 54 (delta 21), reused 39 (delta 9), pack-reused 0
Unpacking objects: 100% (54/54), 2.76 MiB | 5.03 MiB/s, done.


In [ ]:
!pip install -r /content/TiLT-Implementation/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import sys
sys.path.append("/content/TiLT-Implementation/src/")

In [ ]:
from torchvision import transforms
from torchvision.ops import roi_pool
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision.ops import RoIAlign

from dataset import FUNSDDs ## Repo
from datasets import load_dataset

In [ ]:
from transformers import AutoTokenizer
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
hf_ds = load_dataset("nielsr/funsd-layoutlmv3")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset funsd-layoutlmv3 downloaded and prepared to /root/.cache/huggingface/datasets/nielsr___funsd-layoutlmv3/funsd/1.0.0/0e3f4efdfd59aa1c3b4952c517894f7b1fc4d75c12ef01bcc8626a69e41c1bb9. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Lambda(lambda x : 2 * x - 1)])

In [ ]:
train_ds = FUNSDDs(hf_ds['train'],tokenizer = tokenizer, transform = transform)
val_ds = FUNSDDs(hf_ds['test'],tokenizer = tokenizer, transform = transform)

## Modeling

In [ ]:
class ConvBlock(nn.Module):
    """
    A Convolutional Block that consists of two convolution layers each followed by
    instance normalization, LeakyReLU activation and dropout.
    """

    def __init__(self, in_chans: int, out_chans: int, drop_prob: float):
        """
        Args:
            in_chans: Number of channels in the input.
            out_chans: Number of channels in the output.
            drop_prob: Dropout probability.
        """
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.drop_prob = drop_prob

        self.layers = nn.Sequential(
            nn.Conv2d(in_chans, out_chans, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(out_chans),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Dropout2d(drop_prob),
            nn.Conv2d(out_chans, out_chans, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(out_chans),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Dropout2d(drop_prob),
            nn.Conv2d(out_chans, out_chans, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(out_chans),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Dropout2d(drop_prob),
        )

    def forward(self, image: torch.Tensor) -> torch.Tensor:
        """
        Args:
            image: Input 4D tensor of shape `(N, in_chans, H, W)`.
        Returns:
            Output tensor of shape `(N, out_chans, H, W)`.
        """
        return self.layers(image)


class Unet_encoder(nn.Module):

  def __init__(self, 
               in_channels: int = 3,
               channels: int = 32,
               num_pool_layers: int = 4,
               drop_prob: float = 0.0
               ):
    """
        Args:
            in_chans: Number of channels in the input to the U-Net model.
            out_chans: Number of channels in the output to the U-Net model.
            chans: Number of output channels of the first convolution layer.
            num_pool_layers: Number of down-sampling and up-sampling layers.
            drop_prob: Dropout probability.
    """
    super().__init__()

    self.in_channels = in_channels
    self.channels = channels

    self.num_pool_layers = num_pool_layers
    self.drop_prob = drop_prob

    self.down_sample_layers = nn.ModuleList([
        ConvBlock(in_channels, channels, drop_prob)
    ])
    ch = channels

    for _ in range(num_pool_layers - 1):
      self.down_sample_layers.append(ConvBlock(ch, ch*2,drop_prob))
      ch *= 2

    self.conv = ConvBlock(ch, ch*2, drop_prob)

  def forward(self, image: torch.Tensor) -> torch.Tensor:
    '''
    Args:
      Image: Input 4D tensor of shape (Batch Size, in channels, H, W)
    Returns:
      Output tensor of shape (Batch Size, out_channels, H, W)
    '''
    output = image

    ## Appplying down sample layers
    for num, layer in enumerate(self.down_sample_layers):
      output = layer(output)
      output = F.max_pool2d(output, kernel_size = 2, stride = 2, padding = 0)

    output = self.conv(output)  
    return output


class RoIAlign(nn.Module):
  def __init__(self, output_size = (3, 3), spatial_scale = 0.125, sampling_ratio = 2):
    super().__init__()
    # self.output_size = output_size
    # self.spatial_scale = spatial_scale
    # self.sampling_ratio = sampling_ratio
    self.roi_align = RoIAlign(output_size, spatial_scale=spatial_scale, sampling_ratio=sampling_ratio)

  def forward(self, image_embedding, bboxes):
    """
    Args:
      image_embedding: Input 4D tensor of shape (Batch size, in channels, H, W)
      bboxes: Input 3D Tensor of shape (Batch Size, max sequence length, 4) (4 corresponding to xmin, ymin, xmax, ymax)
    Returns:
      feature_map_single_batch: tensor of shape (max sequence length, in channels, *output_size)
    """

    feature_maps_bboxes = []
    for single_batch_img, single_batch_bbox in zip(image_embedding, bboxes):
      feature_map_single_batch = self.roi_align(input = single_batch_img.unsqueeze(0),
                                                rois=torch.cat([torch.zeros(single_batch_bbox.shape[0], 1).to(single_batch_bbox.device), single_batch_bbox], axis = -1).float()
                                                )
      feature_maps_bboxes.append(feature_map_single_batch)

    return torch.stack(feature_maps_bboxes, axis = 0) 

class RoIPool(nn.Module):

  def __init__(self, output_size = (3, 3), spatial_scale = 0.125):
    super().__init__()
    # self.output_size = output_size
    # self.spatial_scale = spatial_scale
    # self.sampling_ratio = sampling_ratio
    self.output_size = output_size
    self.spatial_scale = spatial_scale
    self.roi_pool = roi_pool

  def forward(self, image_embedding, bboxes):
    """
    Args:
      image_embedding: Input 4D tensor of shape (Batch size, in channels, H, W)
      bboxes: Input 3D Tensor of shape (Batch Size, max sequence length, 4) (4 corresponding to xmin, ymin, xmax, ymax)
    Returns:
      feature_map_single_batch: tensor of shape (max sequence length, in channels, *output_size)
    """

    feature_maps_bboxes = []
    for single_batch_img, single_batch_bbox in zip(image_embedding, bboxes):
      feature_map_single_batch = self.roi_pool(input = single_batch_img.unsqueeze(0),
                                                boxes=torch.cat([torch.zeros(single_batch_bbox.shape[0], 1).to(single_batch_bbox.device), single_batch_bbox], axis = -1).float(),
                                                output_size = self.output_size,
                                                spatial_scale = self.spatial_scale
                                                )
      feature_maps_bboxes.append(feature_map_single_batch)

    return torch.stack(feature_maps_bboxes, axis = 0) 

In [ ]:
# Initialization of the UNET encoder
in_channels = 3
num_pool_layers = 3
channels = 16
unet_encoder = Unet_encoder(in_channels = in_channels, channels = channels, num_pool_layers = num_pool_layers)

In [ ]:
# Initialization of the RoI Pooling
output_size = (3,3)
spatial_scale = 48 / 384 # 48/384
sampling_ratio = 2  
# roi_align = RoIAlign(output_size = output_size, spatial_scale = spatial_scale, sampling_ratio = sampling_ratio)
roi_pool = RoIPool(output_size = output_size, spatial_scale = spatial_scale)

In [ ]:
sample_encoding = train_ds[0]
for key in sample_encoding:
  sample_encoding[key].unsqueeze_(0)
  # sample_encoding[key] = torch.stack([sample_encoding[key], sample_encoding[key]], axis = 0)

pixel_values = sample_encoding['pixel_values']
input_ids = sample_encoding['input_ids']
labels = sample_encoding['labels']
attention_mask = sample_encoding['attention_mask']
bboxes = sample_encoding['bboxes']

In [ ]:
# with torch.no_grad():
#   image_embedding = unet_encoder(pixel_values)
# with torch.no_grad():
#   feature_maps_bboxes = roi_pool(image_embedding, bboxes)

In [ ]:
image_embedding = unet_encoder(pixel_values)
feature_maps_bboxes = roi_pool(image_embedding, bboxes)

In [ ]:
feature_maps_bboxes.shape

torch.Size([1, 512, 128, 3, 3])

In [ ]:
sum(i.numel() for i in unet_encoder.parameters()) / 1e6

0.48888

In [ ]:
sum(i.numel() for i in roi_pool.parameters()) / 1e6

0.0